In [62]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [63]:
load_dotenv(override=True)

True

In [64]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/chroma"
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [65]:
loader = PyPDFLoader(file)
file = loader.load()

# Recursive TextSplit

In [66]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [67]:
documents = recur_split.split_documents(file)

# Embbiding (OpenAI)

In [68]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [69]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# LLM (OpenAI)

In [70]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Chain

In [71]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type='mmr'),
)

# Question

In [72]:
question = "O que é Identificar oportunidades?"

# Answer

In [73]:
chat_chain.invoke({"query": question})

{'query': 'O que é Identificar oportunidades?',
 'result': 'Identificar oportunidades é usar a sua imaginação e habilidades para visualizar oportunidades de criação de valor, como a identificação de necessidades e desafios que precisam ser atendidos.'}

# Prompt

In [74]:
prompt = PromptTemplate.from_template(
"""
Você é um assistente educacional que retorna competências empreendedoras.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
**Disciplina**;
**Conteúdo**;
**Objetivo da Aula**.

Com base no documento (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Traga somente 3 competências empreendedoras que estão no documento abaixo e somente do documento (que está no contexto abaixo).
Retorne apenas os nomes das competências, entre a primeira e a segunda separados por vírgula, e entre a segunda e a terceira separada por 'e'. \n\n

Exemplo: \n
Pergunta: "**Disciplina** <Matemática>, sendo o **Conteúdo** <Frações> e o **Objetivo da Aula** é <Calcular juros>." \n
Resposta: Identificar oportunidades, Mobilização de recursos e Educação financeira e econômica. \n\n

Contexto: {context} \n\n
Pergunta: {question}
"""
)

# Chain

In [75]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [ ]:
# question = "**Disciplina** <Matemática>, sendo o **Conteúdo** <Frações> e o **Objetivo da Aula** é <Utilização para calcular taxa de juros>."

# Answer

In [ ]:
# answer = chat_chain.invoke({"query": question})

In [ ]:
# print(answer["result"])

Educação financeira e econômica, Mobilização de pessoas e Visão.
